In [1]:
from experiment import *

2022-06-30 16:02:42.022290: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 16:02:42.044072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 16:02:42.044271: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
seed_sequence_length=200
seed_vocab_size=3000
seed_hidden_units=100
seed_embeddings=2

In [3]:
experiment(sequence_length=seed_sequence_length, vocab_size=seed_vocab_size, hidden_units=seed_hidden_units, embedding_dim=seed_embeddings, verbose=0)

2022-06-30 16:02:45.121224: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-30 16:02:45.162796: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


782/782 [==============================] - 0s 325us/step - loss: 0.2978 - accuracy: 0.8748


(<keras.engine.sequential.Sequential at 0x7f8df8de9fd0>,
 0.8748400211334229)

In [15]:
def next_step (base_sequence_length, base_vocab_size, base_hidden_units, base_embeddings, rate):
    results = {}
    for sequence_length in (base_sequence_length, round (base_sequence_length * rate)):
        for vocab_size in (base_vocab_size, round (base_vocab_size * rate)):
            for hidden_units in (base_hidden_units, round (base_hidden_units)):
                for embeddings in (base_embeddings, 2 * base_embeddings):
                    _, _, accuracy = experiment(sequence_length=sequence_length,
                                                vocab_size=vocab_size,
                                                hidden_units=hidden_units,
                                                embedding_dim=embeddings,
                                                verbose=0)
                    results[(sequence_length, vocab_size, hidden_units, embeddings)] = accuracy
                    
    max_value_key = max(results, key=results.get)
    max_value_accuracy = results[max_value_key]                
    print (f"Maximum Accuracy from Step: {max_value_accuracy} with Key: {max_value_key}")
    return (max_value_key)

In [16]:
next_step(200, 3000, 100, 2, 1.05)

782/782 [==============================] - 0s 323us/step - loss: 0.2946 - accuracy: 0.8768
Maximum Accuracy from Step: 0.8769999742507935 with Key: (200, 3150, 100, 4)


(200, 3150, 100, 4)

In [ ]:
sequence_length=seed_sequence_length
vocab_size=seed_vocab_size
hidden_units=seed_hidden_units
embeddings=seed_embeddings

for _ in range (10):
    sequence_length, vocab_size, hidden_units, embeddings = next_step(sequence_length, vocab_size, hidden_units, embeddings, 1.05)



782/782 [==============================] - 0s 315us/step - loss: 0.2946 - accuracy: 0.8768
Maximum Accuracy from Step: 0.8769999742507935 with Key: (200, 3150, 100, 4)
782/782 [==============================] - 0s 309us/step - loss: 0.2977 - accuracy: 0.8750
Maximum Accuracy from Step: 0.877560019493103 with Key: (210, 3308, 100, 4)
782/782 [==============================] - 0s 324us/step - loss: 0.2964 - accuracy: 0.8753
Maximum Accuracy from Step: 0.8776400089263916 with Key: (220, 3473, 100, 4)
782/782 [==============================] - 0s 327us/step - loss: 0.2937 - accuracy: 0.8764
Maximum Accuracy from Step: 0.8791999816894531 with Key: (231, 3647, 100, 4)
782/782 [==============================] - 0s 310us/step - loss: 0.2922 - accuracy: 0.8794
Maximum Accuracy from Step: 0.8812000155448914 with Key: (243, 3829, 100, 4)
782/782 [==============================] - 0s 319us/step - loss: 0.2905 - accuracy: 0.8813
Maximum Accuracy from Step: 0.8822799921035767 with Key: (255, 3829, 1